# ライブラリの準備

Pytorchのライブラリ(lib)をzip形式でGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5<br>
取得したリンクからidを抜き出す。<br>
**1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5**

In [0]:
# Google Cloud SDKで、Google Driveのファイルを、
# Google Colabolatoryの一時領域にダウンロードするための認証
from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Pytorchのライブラリをダウンロード
id = '1FcWijPxdniRlerqOYq9sSw6qtRX4Ivo5' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
import os
os.chdir('/content')
downloaded.GetContentFile('lib.zip')

In [3]:
# ダウンロードを確認
!pwd
%ls

/content
adc.json  lib.zip  sample_data/


In [4]:
# ライブラリを解凍
%%time
import zipfile
import os
os.chdir('/content')
save_name = "lib"
save_path = save_name + '.zip'
zip = zipfile.ZipFile(save_path) # ZIPファイルを読み込み
zip.extractall() # ZIPファイルを解凍
zip.close() # ZIPファイルをクローズ
os.remove(save_path) # ZIPファイルを消去

CPU times: user 2.05 ms, sys: 2.01 ms, total: 4.06 ms
Wall time: 7.95 ms


In [5]:
# 処理結果を確認
!pwd
!ls
%ls ./lib/

/content
adc.json  lib  sample_data
architecture.py  dataloader.py  network_finetuning.py  optimizer.py  trainer.py
common.py        __init__.py    network.py             support.py


# データセットの準備

## データの格納／展開

データセット(zip)をローカルにダウンロードする。<br>
https://www.kaggle.com/daggysheep/find-a-car-park

ダウンロードしたデータセット(zip)をGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV<br>
取得したリンクからidを抜き出す。<br>
**17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV**

In [6]:
# データセットをダウンロード
%%time
id = '17k1OGlbRgtxvmrL7GjQjhmF6zMVYKGCV' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
os.chdir('/content')
downloaded.GetContentFile('find-a-car-park.zip')

CPU times: user 3.39 s, sys: 14.1 s, total: 17.5 s
Wall time: 45.7 s


In [7]:
# ダウンロードを確認
!pwd
%ls

/content
adc.json  find-a-car-park.zip  lib/  sample_data/


In [19]:
# データセットを解凍
%%time
import zipfile
import os
os.chdir('/content')
save_name = "find-a-car-park"
save_path = save_name + '.zip'
zip = zipfile.ZipFile(save_path) # ZIPファイルを読み込み
zip.extractall(save_name) # ZIPファイルを解凍
zip.close() # ZIPファイルをクローズ
#os.remove(save_path) # ZIPファイルを消去 # 試験中は消さない方が良い

CPU times: user 13.9 s, sys: 3.43 s, total: 17.3 s
Wall time: 30.4 s


In [9]:
# 処理結果を確認
%ls ./find-a-car-park/data

Free/  Full/


In [10]:
# データ個数を確認
!ls -1U "/content/find-a-car-park/data/Free" | wc -l
!ls -1U "/content/find-a-car-park/data/Full" | wc -l

1067
2195


## trainデータ／validationデータの分割

In [11]:
os.chdir('/content/find-a-car-park')
!pwd
%ls

/content/find-a-car-park
data/


In [12]:
# trainデータ、valデータを格納するフォルダを作成
os.mkdir("/content/find-a-car-park/train/")
os.mkdir("/content/find-a-car-park/val/")
!pwd
%ls

/content/find-a-car-park
data/  train/  val/


In [13]:
# フォルダが空であることを確認
!ls "/content/find-a-car-park/train/"
!ls -1U "/content/find-a-car-park/train/" | wc -l
!ls "/content/find-a-car-park/val/"
!ls -1U "/content/find-a-car-park/val/" | wc -l

0
0


In [14]:
#「/content/find-a-car-park/data/」以下の画像を「train」「val」フォルダに振り分ける
# train:val = 3:1
import glob
import shutil
os.chdir('/content/find-a-car-park/data')
label_dirs = os.listdir(path='.')
path = os.getcwd()  

# ラベル名が付与されている各フォルダごとに、中身のファイル名を取り出す
for dir_name in label_dirs:
  files = os.listdir(path + "/" + dir_name) # 1個のラベルフォルダのパスを取得
  numof_data = len(files) # ラベルフォルダ中のデータ数を取得
  files = glob.glob("./" + dir_name + "/*") # ラベルフォルダ中の全ファイルリストを取得

  # ラベルフォルダ中のファイルを抽出して、"val"フォルダに移動する
  # 全体の1/4のデータを"val"フォルダに移動するまで繰り返す
  i = 0
  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    i = i + 1
    if i < numof_data * 1/4:
      data_dir = "/content/find-a-car-park/val/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
    else:
      data_dir = "/content/find-a-car-park/train/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
  shutil.rmtree("./" + dir_name) # データを振り分け終えたラベルフォルダは削除しておく

Number of data in Free: 1067
Number of data in Full: 2195


In [15]:
# データが振り分けられたことを確認
!ls -1U "/content/find-a-car-park/train/Free" | wc -l
!ls -1U "/content/find-a-car-park/train/Full" | wc -l

801
1647


In [16]:
# データが振り分けられたことを確認
!ls -1U "/content/find-a-car-park/val/Free" | wc -l
!ls -1U "/content/find-a-car-park/val/Full" | wc -l

266
548


In [0]:
# [Option] データセットを一旦消す場合に実行
#%rm -r '/content/find-a-car-park'

## [DELETED] Google Drive上でデータセットを解凍して使用する場合

In [0]:
# Google Driveをmountする。
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!unzip '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park.zip'

In [0]:
!ls

In [0]:
!mkdir '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park'

In [0]:
!mv '/content/data' '/content/drive/My Drive/TopSE/ソフトウェア開発実践演習/src/pytorch/find-a-car-park'

# 学習／検証

In [20]:
%%time
os.chdir('/content/')
!npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.909s
     |████████████████████████████████| 686kB 5.0MB/s 
     |████████████████████████████████| 204kB 18.1MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=0a44534528d7b22568591f3b6de1e9a987b4a340a36de64baeed9d429d2371c4
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=96bb8cc15618483f9e8f05be6bc15c45fea421926604dfee25a0d3794c053436
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 160 ms, sys: 252 ms, total: 412 ms
Wall time: 17.5 s


In [21]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 21.2 ms, sys: 29.9 ms, total: 51.1 ms
Wall time: 6.14 s


In [22]:
!pwd
!ls

/content
adc.json	 find-a-car-park.zip  sample_data  visdomlog.txt
find-a-car-park  lib		      url.txt


In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [24]:
# アーキテクチャを選択して使用する。
%%time
#net = network.VGG16(class_size=2)
net = network.InceptionV3(class_size=2)
#net = network.ResNet50(class_size=2)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [0]:
%%time
batch_size = 128
# フォルダのパスはかえる
train_dataset = torchvision.datasets.ImageFolder(root='/content/find-a-car-park/train/', transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.ImageFolder(root='/content/find-a-car-park/val/', transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size)

In [0]:
%%time
model = architecture.CNN_Architecture(net)

In [0]:
%%time
model.train(data_loader, epoch_count=50, is_inception=False)

In [0]:
%%time
model.predict(data_loader['val'])

------------------